# Resource usage of the StellarGraph class

> This notebooks records the time and memory (both peak and long-term) required to construct a StellarGraph object for several datasets.

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/zzz-internal-developers/graph-resource-usage.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/zzz-internal-developers/graph-resource-usage.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>

This notebook is aimed at helping contributors to the StellarGraph library itself understand how their changes affect the resource usage of the `StellarGraph` object.

Various measures of resource usage for several "real world" graphs of various sizes are recorded:

- time for construction
- memory usage of the final `StellarGraph` object
- peak memory usage during `StellarGraph` construction (both absolute, and additional compared to the raw input data)

These are recorded both with explicit nodes (and node features if they exist), and implicit/inferred nodes.

The memory usage is recorded end-to-end. That is, the recording starts from data on disk and continues until the `StellarGraph` object has been constructed and other data has been cleaned up. This is important for accurately recording the total memory usage, as NumPy arrays can often share data with existing arrays in memory and so retroactive or partial (starting from data in memory) analysis can miss significant amounts of data. The parsing code in `stellargraph.datasets` doesn't allow determining the memory usage of the intermediate nodes and edges input to the `StellarGraph` constructor, and so cannot be used here.

In [1]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.1.0b

In [2]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.1.0b")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.1.0b, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [3]:
import stellargraph as sg
import pandas as pd
import numpy as np

import gc
import json
import os
import timeit
import tempfile
import tracemalloc

## Optional reddit data

The original GraphSAGE paper evaluated on a reddit dataset, available at <http://snap.stanford.edu/graphsage/#datasets>. This dataset is large (1.3GB compressed) and so there is not automatic download support for it. The following `reddit_path` variable controls whether and how the reddit dataset is included:

- to ignore the dataset: set the variable to `None`
- to include the dataset: download the dataset zip, decompress it, and set the variable to the decompressed directory

In [4]:
reddit_path = os.path.expanduser("~/data/reddit")

## Datasets

### Cora

In [5]:
cora = sg.datasets.Cora()
cora.download()

cora_cites_path = os.path.join(cora.data_directory, "cora.cites")
cora_content_path = os.path.join(cora.data_directory, "cora.content")
cora_dtypes = {0: int, **{i: np.float32 for i in range(1, 1433 + 1)}}


def cora_pandas_parts(include_nodes):
    if include_nodes:
        nodes = pd.read_csv(
            cora_content_path,
            header=None,
            sep="\t",
            index_col=0,
            usecols=range(0, 1433 + 1),
            dtype=cora_dtypes,
            na_filter=False,
        )
    else:
        nodes = None
    edges = pd.read_csv(
        cora_cites_path,
        header=None,
        sep="\t",
        names=["source", "target"],
        dtype=int,
        na_filter=False,
    )
    return nodes, edges, {}


def cora_indexed_array_parts(include_nodes):
    nodes, edges, args = cora_pandas_parts(include_nodes)
    if nodes is not None:
        nodes = sg.IndexedArray(nodes.to_numpy(), index=nodes.index)
    return nodes, edges, args

### BlogCatalog3

In [6]:
blogcatalog3 = sg.datasets.BlogCatalog3()
blogcatalog3.download()

blogcatalog3_edges = os.path.join(blogcatalog3.data_directory, "edges.csv")
blogcatalog3_group_edges = os.path.join(blogcatalog3.data_directory, "group-edges.csv")
blogcatalog3_groups = os.path.join(blogcatalog3.data_directory, "groups.csv")
blogcatalog3_nodes = os.path.join(blogcatalog3.data_directory, "nodes.csv")


def blogcatalog3_parts(include_nodes):
    if include_nodes:
        raw_nodes = pd.read_csv(blogcatalog3_nodes, header=None)[0]
        raw_groups = pd.read_csv(blogcatalog3_groups, header=None)[0]
        nodes = {
            "user": pd.DataFrame(index=raw_nodes),
            "group": pd.DataFrame(index=-raw_groups),
        }
    else:
        nodes = None

    edges = pd.read_csv(blogcatalog3_edges, header=None, names=["source", "target"])

    group_edges = pd.read_csv(
        blogcatalog3_group_edges, header=None, names=["source", "target"]
    )
    group_edges["target"] *= -1
    start = len(edges)
    group_edges.index = range(start, start + len(group_edges))

    edges = {"friend": edges, "belongs": group_edges}
    return nodes, edges, {}

### FB15k

In [7]:
fb15k = sg.datasets.FB15k()
fb15k.download()
fb15k_files = [
    os.path.join(fb15k.data_directory, f"freebase_mtr100_mte100-{x}.txt")
    for x in ["train", "test", "valid"]
]


def fb15k_parts(include_nodes, usecols=None):
    loaded = [
        pd.read_csv(
            name,
            header=None,
            names=["source", "label", "target"],
            sep="\t",
            dtype=str,
            na_filter=False,
            usecols=usecols,
        )
        for name in fb15k_files
    ]
    edges = pd.concat(loaded, ignore_index=True)

    if include_nodes:
        # infer the set of nodes manually, in a memory-minimal way
        raw_nodes = set(edges.source)
        raw_nodes.update(edges.target)
        nodes = pd.DataFrame(index=raw_nodes)
    else:
        nodes = None

    return nodes, edges, {"edge_type_column": "label"}


def fb15k_no_edge_types_parts(include_nodes):
    nodes, edges, _ = fb15k_parts(include_nodes, usecols=["source", "target"])
    return nodes, edges, {}

### Reddit

As discussed above, the reddit dataset is large and optional. It is also slow to parse, as the graph structure is a huge JSON file. Thus, we prepare the dataset by converting that JSON file into a NumPy edge list array, of shape `(num_edges, 2)`. This is significantly faster to load from disk.

In [8]:
%%time

# if requested, prepare the reddit dataset by saving the slow-to-read JSON to a temporary .npy file
if reddit_path is not None:
    reddit_graph_path = os.path.join(reddit_path, "reddit-G.json")
    reddit_feats_path = os.path.join(reddit_path, "reddit-feats.npy")

    with open(reddit_graph_path) as f:
        reddit_g = json.load(f)
    reddit_numpy_edges = np.array([[x["source"], x["target"]] for x in reddit_g["links"]])
    
    reddit_edges_file = tempfile.NamedTemporaryFile(suffix=".npy")
    np.save(reddit_edges_file, reddit_numpy_edges)

CPU times: user 16.2 s, sys: 1.99 s, total: 18.2 s
Wall time: 18.2 s


In [9]:
def reddit_numpy_parts(include_nodes):
    if include_nodes:
        nodes = np.load(reddit_feats_path).astype(np.float32)
    else:
        nodes = None

    raw_edges = np.load(reddit_edges_file.name)
    edges = pd.DataFrame(raw_edges, columns=["source", "target"])
    return nodes, edges, {}


def reddit_pandas_parts(include_nodes):
    nodes, edges, args = reddit_numpy_parts(include_nodes)
    if nodes is not None:
        nodes = pd.DataFrame(nodes)

    return nodes, edges, args

### Collected

In [10]:
datasets = {
    "Cora (Pandas)": cora_pandas_parts,
    "Cora (IndexedArray)": cora_indexed_array_parts,
    "BlogCatalog3": blogcatalog3_parts,
    "FB15k (no edge types)": fb15k_no_edge_types_parts,
    "FB15k": fb15k_parts,
}
if reddit_path is not None:
    datasets["reddit (Pandas)"] = reddit_pandas_parts
    datasets["reddit (NumPy)"] = reddit_numpy_parts

## Measurement

In [11]:
def mem_snapshot_diff(after, before):
    """Total memory difference between two tracemalloc.snapshot objects"""
    return sum(elem.size_diff for elem in after.compare_to(before, "lineno"))

In [12]:
# names of columns computed by the measurement code
def measurement_columns(title):
    names = [
        "time",
        "memory (graph)",
        "memory (graph, not shared with data)",
        "peak memory (graph)",
        "peak memory (graph, ignoring data)",
        "memory (data)",
        "peak memory (data)",
    ]
    return [(title, x) for x in names]


columns = pd.MultiIndex.from_tuples(
    [
        ("graph", "nodes"),
        ("graph", "node feat size"),
        ("graph", "edges"),
        *measurement_columns("explicit nodes"),
        *measurement_columns("inferred nodes (no features)"),
    ]
)

In [13]:
def measure_time(f, include_nodes):
    nodes, edges, args = f(include_nodes)
    start = timeit.default_timer()
    sg.StellarGraph(nodes, edges, **args)
    end = timeit.default_timer()
    return end - start

In [14]:
def measure_memory(f, include_nodes):
    """
    Measure exactly what it takes to load the data.
    
    - the size of the original edge data (as a baseline)
    - the size of the final graph
    - the peak memory use of both
    
    This uses a similar technique to the 'allocation_benchmark' fixture in tests/test_utils/alloc.py.
    """
    gc.collect()
    # ensure we're measuring the worst-case peak, when no GC happens
    gc.disable()

    tracemalloc.start()
    snapshot_start = tracemalloc.take_snapshot()

    nodes, edges, args = f(include_nodes)

    gc.collect()
    _, data_memory_peak = tracemalloc.get_traced_memory()
    snapshot_data = tracemalloc.take_snapshot()

    if include_nodes:
        assert nodes is not None, f
        sg_g = sg.StellarGraph(nodes, edges, **args)
    else:
        assert nodes is None, f
        sg_g = sg.StellarGraph(edges=edges, **args)

    gc.collect()
    snapshot_graph = tracemalloc.take_snapshot()

    # clean up the input data and anything else leftover, so that the snapshot
    # includes only the long-lasting data: the StellarGraph.
    del edges
    del nodes
    del args
    gc.collect()

    _, graph_memory_peak = tracemalloc.get_traced_memory()
    snapshot_end = tracemalloc.take_snapshot()
    tracemalloc.stop()

    gc.enable()

    data_memory = mem_snapshot_diff(snapshot_data, snapshot_start)
    graph_memory = mem_snapshot_diff(snapshot_end, snapshot_start)
    graph_over_data_memory = mem_snapshot_diff(snapshot_graph, snapshot_data)

    return (
        sg_g,
        graph_memory,
        graph_over_data_memory,
        graph_memory_peak,
        graph_memory_peak - data_memory,
        data_memory,
        data_memory_peak,
    )

In [15]:
def measure(f):
    time_nodes = measure_time(f, include_nodes=True)
    time_no_nodes = measure_time(f, include_nodes=False)

    sg_g, *mem_nodes = measure_memory(f, include_nodes=True)
    _, *mem_no_nodes = measure_memory(f, include_nodes=False)

    feat_sizes = sg_g.node_feature_sizes()
    try:
        feat_sizes = feat_sizes[sg_g.unique_node_type()]
    except ValueError:
        pass

    return [
        sg_g.number_of_nodes(),
        feat_sizes,
        sg_g.number_of_edges(),
        time_nodes,
        *mem_nodes,
        time_no_nodes,
        *mem_no_nodes,
    ]

In [16]:
%%time
recorded = [measure(f) for f in datasets.values()]

CPU times: user 28.6 s, sys: 7.12 s, total: 35.7 s
Wall time: 35.9 s


In [17]:
raw = pd.DataFrame(recorded, columns=columns, index=datasets.keys())
raw

graph                                     \
                        nodes           node feat size     edges   
Cora (Pandas)            2708                     1433      5429   
Cora (IndexedArray)      2708                     1433      5429   
BlogCatalog3            10351  {'group': 0, 'user': 0}    348459   
FB15k (no edge types)   14951                        0    592213   
FB15k                   14951                        0    592213   
reddit (Pandas)        232965                      602  11606919   
reddit (NumPy)         232965                      602  11606919   

                      explicit nodes                 \
                                time memory (graph)   
Cora (Pandas)               0.026571       15629802   
Cora (IndexedArray)         0.001719       15628450   
BlogCatalog3                0.022646        7464151   
FB15k (no edge types)       0.114241        8767710   
FB15k                       0.666503       14591290   
reddit (Pandas)             3.174843      758540997   
reddit (NumPy)              0.560628      758540389   

                                                            \
                      memory (graph, not shared with data)   
Cora (Pandas)                                     15609081   
Cora (IndexedArray)                                  84249   
BlogCatalog3                                      10257710   
FB15k (no edge types)                              7782128   
FB15k                                             18196549   
reddit (Pandas)                                  758547824   
reddit (NumPy)                                   197562200   

                                                                              \
                      peak memory (graph) peak memory (graph, ignoring data)   
Cora (Pandas)                    46764049                           31079400   
Cora (IndexedArray)              31995057                           16357628   
BlogCatalog3                     18374776                           12704583   
FB15k (no edge types)            27616406                           11936303   
FB15k                            62450591                           41547764   
reddit (Pandas)                1868696353                         1121990320   
reddit (NumPy)                 1682947017                          936252548   

                                                        \
                      memory (data) peak memory (data)   
Cora (Pandas)              15684649           31995281   
Cora (IndexedArray)        15637429           31995057   
BlogCatalog3                5670193           10805733   
FB15k (no edge types)      15680103           25831842   
FB15k                      20902827           35792614   
reddit (Pandas)           746706033         1682947017   
reddit (NumPy)            746694469         1682947017   

                      inferred nodes (no features)                 \
                                              time memory (graph)   
Cora (Pandas)                             0.002871         116170   
Cora (IndexedArray)                       0.002337         116170   
BlogCatalog3                              0.027026        7463063   
FB15k (no edge types)                     0.213923        8767742   
FB15k                                     0.758157       14591498   
reddit (Pandas)                           0.505142      200336039   
reddit (NumPy)                            0.484281      200336071   

                                                            \
                      memory (graph, not shared with data)   
Cora (Pandas)                                       118537   
Cora (IndexedArray)                                 118537   
BlogCatalog3                                      10257062   
FB15k (no edge types)                              7904840   
FB15k                                             18320437   
reddit (Pandas)                                  2003376

## Pretty results

This shows the results in a prettier way, such as memory in MB instead of bytes.

In [18]:
mem_columns = raw.columns[["memory" in x[1] for x in raw.columns]]

memory_mb = raw.copy()
memory_mb[mem_columns] = (memory_mb[mem_columns] / 10 ** 6).round(3)
memory_mb

graph                                     \
                        nodes           node feat size     edges   
Cora (Pandas)            2708                     1433      5429   
Cora (IndexedArray)      2708                     1433      5429   
BlogCatalog3            10351  {'group': 0, 'user': 0}    348459   
FB15k (no edge types)   14951                        0    592213   
FB15k                   14951                        0    592213   
reddit (Pandas)        232965                      602  11606919   
reddit (NumPy)         232965                      602  11606919   

                      explicit nodes                 \
                                time memory (graph)   
Cora (Pandas)               0.026571         15.630   
Cora (IndexedArray)         0.001719         15.628   
BlogCatalog3                0.022646          7.464   
FB15k (no edge types)       0.114241          8.768   
FB15k                       0.666503         14.591   
reddit (Pandas)             3.174843        758.541   
reddit (NumPy)              0.560628        758.540   

                                                            \
                      memory (graph, not shared with data)   
Cora (Pandas)                                       15.609   
Cora (IndexedArray)                                  0.084   
BlogCatalog3                                        10.258   
FB15k (no edge types)                                7.782   
FB15k                                               18.197   
reddit (Pandas)                                    758.548   
reddit (NumPy)                                     197.562   

                                                                              \
                      peak memory (graph) peak memory (graph, ignoring data)   
Cora (Pandas)                      46.764                             31.079   
Cora (IndexedArray)                31.995                             16.358   
BlogCatalog3                       18.375                             12.705   
FB15k (no edge types)              27.616                             11.936   
FB15k                              62.451                             41.548   
reddit (Pandas)                  1868.696                           1121.990   
reddit (NumPy)                   1682.947                            936.253   

                                                        \
                      memory (data) peak memory (data)   
Cora (Pandas)                15.685             31.995   
Cora (IndexedArray)          15.637             31.995   
BlogCatalog3                  5.670             10.806   
FB15k (no edge types)        15.680             25.832   
FB15k                        20.903             35.793   
reddit (Pandas)             746.706           1682.947   
reddit (NumPy)              746.694           1682.947   

                      inferred nodes (no features)                 \
                                              time memory (graph)   
Cora (Pandas)                             0.002871          0.116   
Cora (IndexedArray)                       0.002337          0.116   
BlogCatalog3                              0.027026          7.463   
FB15k (no edge types)                     0.213923          8.768   
FB15k                                     0.758157         14.591   
reddit (Pandas)                           0.505142        200.336   
reddit (NumPy)                            0.484281        200.336   

                                                            \
                      memory (graph, not shared with data)   
Cora (Pandas)                                        0.119   
Cora (IndexedArray)                                  0.119   
BlogCatalog3                                        10.257   
FB15k (no edge types)                                7.905   
FB15k                                               18.320   
reddit (Pandas)                                    200.3

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/zzz-internal-developers/graph-resource-usage.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/zzz-internal-developers/graph-resource-usage.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>